In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from classical_approach import *

In [3]:
with open(halos_path, "rb") as f:
  particles, halo_slice  = pickle.load(f)

train_indices, test_indices = train_test_split(jnp.arange(len(particles)), test_size=0.5, random_state=42)

labels = jnp.log10(
    halo_slice["GroupStellarMass"] * 1e10 / 0.677
)  # label by logMstar
logc = jnp.log10(
    halo_slice["SubhaloVmax"]
    / jnp.sqrt(G * halo_slice["Group_M_Mean200"] / halo_slice["Group_R_Mean200"])
)
logm = jnp.log10(jnp.array([particle["count"] for particle in particles]))


train_labels, test_labels = labels[train_indices], labels[test_indices]
train_logc, test_logc = logc[train_indices], logc[test_indices]
train_logm, test_logm = logm[train_indices], logm[test_indices]

In [4]:
train_x = jnp.stack([train_logc, train_logm], axis=-1)
test_x = jnp.stack([test_logc, test_logm], axis=-1)
mean_x = train_x.mean(axis=0, keepdims=True)
std_x = train_x.std(axis=0, keepdims=True)

train_transformed = (train_x - mean_x) / std_x
test_transformed = (test_x - mean_x) / std_x